In [1]:
import numpy as np 
import math 
import rospy 
from tf import transformations
from std_msgs.msg import Header, ColorRGBA
from visualization_msgs.msg import Marker, MarkerArray
import sys
sys.path.append("..")
from kinematics.structure import CHAIN
from kinematics.robot import RobotClass
from kinematics.utils.util_ik import add_joints, find_route, make_ik_input
from kinematics.utils.util_rviz import publish_viz_robot, publish_viz_markers
from kinematics.utils.util_structure import *
print("Done.")

Done.


In [2]:
class RvizClass:
    def __init__(self, file_name = "../urdf/ur5e/ril_robot.urdf", base_offset=[0,0,0]):
        rospy.init_node("Run_Robot")
        self.pub_robot      = rospy.Publisher('viz_robot', MarkerArray, queue_size=10)
        self.pub_obj     = rospy.Publisher('viz_objs', MarkerArray, queue_size=10)
        self.chain          = CHAIN(file_name=file_name, base_offset=base_offset, verbose=False)
        self.chain.add_joi_to_robot()
        self.chain.add_link_to_robot()
        self.ctrl_joint_num = 7

    def check_fk(self, q_list):
        update_q_chain(self.chain.joint, q_list, self.ctrl_joint_num)
        self.chain.fk_chain(1)
        p_list     = get_p_chain(self.chain.joint)
        return p_list 

    def publish_robot(self, q_list):
        update_q_chain(self.chain.joint, q_list, self.ctrl_joint_num)
        self.chain.fk_chain(1)
        p_list     = get_p_chain(self.chain.joint)
        R_list     = get_R_chain(self.chain.joint)
        rpy_list   = get_rpy_from_R_mat(R_list)
        mesh_list  = get_mesh_chain(self.chain.link)
        scale_list = get_scale(self.chain.link)
        color_list = get_link_color(self.chain.link)
        viz_links  =  get_viz_ingredients(p_list, rpy_list, mesh_list, scale_list, color_list)
        viz_trg_robot = publish_viz_robot(viz_links)
        self.pub_robot.publish(viz_trg_robot)

    def publish_markers(self, obj):
        viz_obj = publish_viz_markers(obj)
        self.pub_obj.publish(viz_obj)

def make_markers(name, type, pos, rot, size, color): 
    return {"name":name, "type":type, "info":pos+rot+size, "color":color}


### Inverse Kinematics Demo

In [3]:
file_name = "../urdf/ur5e/ril_robot.urdf"
base_offset = [0.18,0.,0.79]
robot     = RobotClass(file_name=file_name, base_offset=base_offset)
rviz = RvizClass(file_name=file_name, base_offset=base_offset)

# q = robot.solve_ik(target_name = ['wrist_3_joint', 'mount_camera_joint'],
#             target_position  = [[0.3,0.,1.3],[0,0,0]],
#             target_rotation  = [[0,0,0],[-np.deg2rad(-40), 0, +np.deg2rad(90)]],
#             solve_position   = [1,0],
#             solve_rotation   = [0,1],
#             weight_position  = 1,
#             weight_rotation  = 1,
#             joi_ctrl_num= 6)

q_list = robot.solve_ik(target_name = ['mount_camera_joint'],
            target_position  = [[0.41,0.,1.3]],
            target_rotation  = [[-np.deg2rad(40), +np.deg2rad(180), +np.deg2rad(270)]],
            solve_position   = [1],
            solve_rotation   = [1],
            weight_position  = 1,
            weight_rotation  = 1,
            joi_ctrl_num= 6)

# q = robot.solve_ik(target_name = ['wrist_3_joint', 'gripper_tcp_joint'],
#             target_position  = [[0.35,0.,1.3],[0,0,0]],
#             target_rotation  = [[0,0,0],[0, +np.deg2rad(40)+np.deg2rad(90), 0]],
#             solve_position   = [1,0],
#             solve_rotation   = [0,1],
#             weight_position  = 1,
#             weight_rotation  = 1,
#             joi_ctrl_num= 6)

# q         = robot.solve_ik(target_name = ['gripper_tcp_joint'],
#             target_position  = [[0.4+0.18,0.,1.4]],
#             target_rotation  = [[0, -math.pi/4-math.pi, 0]],
#             solve_position   = [1],
#             solve_rotation   = [1],
#             weight_position  = 1,
#             weight_rotation  = 1,
#             joi_ctrl_num= 6)

# q_list = np.array([0,0,0,0,0,0,0])#q#np.array([0, 0, -math.pi, math.pi/2, -math.pi/2, -math.pi/2, 0])
#q_list[6] += math.pi/2
print("Joint Value: {}".format(q_list))
rviz.publish_robot(q_list=q_list)
print("Control Joint: {}/Shape: {}".format(q_list, q_list.shape))

print("Position of {}: {}".format(robot.chain.joint[1].name, [round(robot.chain.joint[1].p.T[0][0],3),
                                                              round(robot.chain.joint[1].p.T[0][1],3), 
                                                              round(robot.chain.joint[1].p.T[0][2],3)]))
                                                              
print("Position of {}: {}".format(robot.chain.joint[6].name, [round(robot.chain.joint[6].p.T[0][0],3),
                                                              round(robot.chain.joint[6].p.T[0][1],3), 
                                                              round(robot.chain.joint[6].p.T[0][2],3)]))

print("Position of {}: {}".format(robot.chain.joint[8].name, [round(robot.chain.joint[8].p.T[0][0],3),
                                                              round(robot.chain.joint[8].p.T[0][1],3), 
                                                              round(robot.chain.joint[8].p.T[0][2],3)]))

print("Position of {}: {}".format(robot.chain.joint[10].name, [round(robot.chain.joint[10].p.T[0][0],3),
                                                              round(robot.chain.joint[10].p.T[0][1],3), 
                                                              round(robot.chain.joint[10].p.T[0][2],3)]))
                                                              

Joint Value: [ 0.         -1.5707964  -2.31060578  2.14308639  1.73831577  0.87425728
 -1.57079642]
Control Joint: [ 0.         -1.5707964  -2.31060578  2.14308639  1.73831577  0.87425728
 -1.57079642]/Shape: (7,)
Position of shoulder_pan_joint: [0.18, 0.0, 0.953]
Position of wrist_3_joint: [0.314, -0.0, 1.332]
Position of mount_camera_joint: [0.454, -0.0, 1.3]
Position of gripper_tcp_joint: [0.497, -0.0, 1.114]


In [4]:
tcp_position    = rviz.chain.joint[10].p.reshape(3,).tolist()
tcp_rotation =  decompose_rotation_matrix(rviz.chain.joint[10].R).tolist()
camera_position = rviz.chain.joint[8].p.reshape(3,).tolist()
camera_rotation = decompose_rotation_matrix(rviz.chain.joint[8].R).tolist()

obs_info  = make_markers(name="tcp_marker", 
                         type="cube",  
                         pos=tcp_position, 
                         rot=tcp_rotation, 
                         size=[0.025, 0.025, 0.025], 
                         color=[1,0,0,0.5])

obs_info2  = make_markers(name="camera_marker", 
                         type="cube",  
                         pos=camera_position, 
                         rot=camera_rotation, 
                         size=[0.090, 0.025, 0.025], 
                         color=[0,1,0,1])

obs_info3  = make_markers(name="RobotTable", 
                         type="cube",  
                         pos=[0,0,0.79/2], 
                         rot=[0,0,0], 
                         size=[0.76,0.72,0.79], 
                         color=[0,0,0,0.2])

obs_info4  = make_markers(name="Table", 
                         type="cube",  
                         pos=[0.76/2+1.2/2,0,0.74/2], 
                         rot=[0,0,0], 
                         size=[1.2,0.9,0.74], 
                         color=[0,0,0,0.2])
obs_info_lst = [obs_info, obs_info2, obs_info3, obs_info4]        

START = True

""" VISUALIZE """
if START:
    rendering=0
    rate = rospy.Rate(50)
    while not rospy.is_shutdown():
        if rendering == 1000:
            break 
        rviz.publish_robot(q_list=q_list)
        rviz.publish_markers(obs_info_lst)
        rendering +=1
        rate.sleep()


### Joint Configuration Visualization 

In [5]:
"""
base_joint                         =  rviz.chain.joint[0].p.reshape(3,).tolist()#0
shoulder_pan_joint                 =  rviz.chain.joint[1].p.reshape(3,).tolist()#1
shoulder_lift_joint                =  rviz.chain.joint[2].p.reshape(3,).tolist()#2
elbow_joint                        =  rviz.chain.joint[3].p.reshape(3,).tolist()#3
wrist_1_joint                      =  rviz.chain.joint[4].p.reshape(3,).tolist()#4
wrist_2_joint                      =  rviz.chain.joint[5].p.reshape(3,).tolist()#5 
wrist_3_joint                      =  rviz.chain.joint[6].p.reshape(3,).tolist()#6 
mount_joint                        =  rviz.chain.joint[7].p.reshape(3,).tolist()#7
mount_camera_joint                 =  rviz.chain.joint[8].p.reshape(3,).tolist()#8  
gripper_base_joint                 =  rviz.chain.joint[9].p.reshape(3,).tolist()#9 
gripper_tcp_joint                  =  rviz.chain.joint[10].p.reshape(3,).tolist()#10
gripper_finger1_joint              =  rviz.chain.joint[11].p.reshape(3,).tolist()#11
gripper_finger1_inner_knuckle_joint=  rviz.chain.joint[12].p.reshape(3,).tolist()#12
gripper_finger1_finger_tip_joint   =  rviz.chain.joint[13].p.reshape(3,).tolist()#13
gripper_finger2_joint              =  rviz.chain.joint[14].p.reshape(3,).tolist()#14
gripper_finger2_inner_knuckle_joint=  rviz.chain.joint[15].p.reshape(3,).tolist()#15
gripper_finger2_finger_tip_joint   =  rviz.chain.joint[16].p.reshape(3,).tolist()#16
""" 

joint_info_lst = []
p_list     = get_p_chain(rviz.chain.joint)
R_list     = get_R_chain(rviz.chain.joint)
name_list  = get_name_chain(rviz.chain.joint)
rpy_list   = get_rpy_from_R_mat(R_list) 

for idx, (position, rpy, name) in enumerate(zip(p_list, rpy_list, name_list)): 
    # Robot&Camera Part
    if idx <10:
        joint_info = make_markers(name=str(name), 
                                type="sphere",  
                                pos=position.reshape(3,).tolist(), 
                                rot=rpy.tolist(), 
                                size=[0.05, 0.05, 0.05], 
                                color=[1,0,0,1])
    # Gripper Part 
    else: 
        joint_info = make_markers(name=str(name), 
                                type="sphere",  
                                pos=position.reshape(3,).tolist(), 
                                rot=rpy.tolist(), 
                                size=[0.01, 0.01, 0.01], 
                                color=[1,0,0,1])
    joint_info_lst.append(joint_info)
    

JOINTCONFIG_VIZ= True 
if JOINTCONFIG_VIZ:
    rendering=0
    rate = rospy.Rate(50)
    while not rospy.is_shutdown():
        if rendering == 1000:
            break 
        rviz.publish_robot(q_list=q_list)
        rviz.publish_markers(joint_info_lst)
        rendering +=1
        rate.sleep()
print("Done.")

### Link Configuration Visualization

In [ ]:
link_info_lst = []
cap_R_list = get_cap_R_chain(rviz.chain.link)
rpy_list = get_rpy_from_R_mat(cap_R_list)
link_p_list = get_center_p_chain(rviz.chain.link)
name_list  = get_name_chain(rviz.chain.link) 

for idx, (position, rpy, name) in enumerate(zip(link_p_list, rpy_list, name_list)): 
    # Robot&Camera Part
    if idx <10:
        link_info = make_markers(name=str(name), 
                                type="sphere",  
                                pos=position.reshape(3,).tolist(), 
                                rot=rpy.tolist(), 
                                size=[0.025, 0.025, 0.025], 
                                color=[0,1,0,1])
    # Gripper Part 
    else: 
        link_info = make_markers(name=str(name), 
                                type="sphere",  
                                pos=position.reshape(3,).tolist(), 
                                rot=rpy.tolist(), 
                                size=[0.01, 0.01, 0.01], 
                                color=[0,1,0,1])
    link_info_lst.append(link_info)
    

LINKCONFIG_VIZ= True 
if LINKCONFIG_VIZ:
    rendering=0
    rate = rospy.Rate(50)
    while not rospy.is_shutdown():
        if rendering == 2000:
            break 
        rviz.publish_robot(q_list=q_list)
        rviz.publish_markers(link_info_lst)
        rendering +=1
        rate.sleep()
print("Done.")

### Capsulation

In [ ]:
link_info_lst = []
cap_R_list = get_cap_R_chain(rviz.chain.link)
rpy_list = get_rpy_from_R_mat(cap_R_list)
link_p_list = get_center_p_chain(rviz.chain.link)
name_list  = get_name_chain(rviz.chain.link) 
height_list = get_height_chain(rviz.chain.link)
radius_list = get_cap_radius(rviz.chain.link)
size_list = get_cap_size_chain(rviz.chain.link)

for idx, (position, rpy, name, height, radius) in enumerate(zip(link_p_list, rpy_list, name_list, height_list, radius_list)): 
    link_info = make_markers(name=str(name), 
                            type="sphere",  
                            pos=position.reshape(3,).tolist(), 
                            rot=rpy.tolist(), 
                            size=[radius, radius, height], 
                            color=[0,1,0,1])

    link_info_lst.append(link_info)
    

CAPSULECONFIG_VIZ= False 
if CAPSULECONFIG_VIZ:
    rendering=0
    rate = rospy.Rate(50)
    while not rospy.is_shutdown():
        if rendering == 1000:
            break 
        rviz.publish_robot(q_list=q_list)
        rviz.publish_markers(link_info_lst)
        rendering +=1
        rate.sleep()
print("Done.")